In [ ]:
import sys
sys.path.append('..')
from src.includes import *

model_name = 'bplstm'
home_path = '/home/swj/VD/'
embedding_path = home_path + 'embedding/'

#default mode / type of vulnerability
mode = "command_injection"
data_path = home_path + 'data/python_data_set/' + mode + '/'

#get the vulnerability from the command line argument


progress = 0
count = 0


### paramters for the filtering and creation of samples
#restriction = [20000,5,6,10] #which samples to filter out
restriction = [10000,5,3,5] #which samples to filter out

step = 5 #step lenght n in the description
fulllength = 200 #context length m in the description

mode2 = str(step)+"_"+str(fulllength) 

### hyperparameters for the w2v model
mincount = 10 #minimum times a word has to appear in the corpus to be in the word2vec model
iterationen = 300 #training iterations for the word2vec model
s = 200 #dimensions of the word2vec model
w = "withString" #word2vec model is not replacing strings but keeping them

#get word2vec model
w2v = embedding_path + "word2vec_"+w+str(mincount) + "-" + str(iterationen) +"-" + str(s)
w2vmodel = w2v + ".model"

#load word2vec model
if not (os.path.isfile(w2vmodel)):
    print("word2vec model is still being created...")
    sys.exit()


w2v_model = Word2Vec.load(w2vmodel)
print('word2vec model loaded!')
word_vectors = w2v_model.wv

#load data
with open(data_path + 'plain_' + mode, 'r') as infile:
    data = json.load(infile)
    
now = datetime.now() # current date and time
nowformat = now.strftime("%H:%M")
print("finished loading. ", nowformat)

allblocks = []

for r in data:
    progress = progress + 1

    for c in data[r]:

        if "files" in data[r][c]:                      
            if len(data[r][c]["files"]) > restriction[3]:
                #too many files
                continue

            for f in data[r][c]["files"]:

                if len(data[r][c]["files"][f]["changes"]) >= restriction[2]:
                    #too many changes in a single file
                    continue

                if not "source" in data[r][c]["files"][f]:
                    #no sourcecode
                    continue

                if "source" in data[r][c]["files"][f]:
                    sourcecode = data[r][c]["files"][f]["source"]                          
                    if len(sourcecode) > restriction[0]:
                        #sourcecode is too long
                        continue

                allbadparts = []

                for change in data[r][c]["files"][f]["changes"]:

                    #get the modified or removed parts from each change that happened in the commit                  
                    badparts = change["badparts"]
                    count = count + len(badparts)

                    if len(badparts) > restriction[1]:
                        #too many modifications in one change
                        break

                    for bad in badparts:
                        #check if they can be found within the file
                        pos = myutils.findposition(bad,sourcecode)
                        if not -1 in pos:
                            allbadparts.append(bad)

                    if (len(allbadparts) > restriction[2]):
                        #too many bad positions in the file
                        break

                if(len(allbadparts) > 0):
                    if len(allbadparts) < restriction[2]:
                        #find the positions of all modified parts
                        positions = myutils.findpositions(allbadparts,sourcecode)

                        #get the file split up in samples
                        blocks = myutils.getblocks(sourcecode, positions, step, fulllength)

                        for b in blocks:
                          #each is a tuple of code and label
                            allblocks.append(b)

keys = []

#randomize the sample and split into train, validate and final test set
for i in range(len(allblocks)):
    keys.append(i)
random.shuffle(keys)

print('there are total ' + str(len(keys)) + ' files')
cutoff = round(0.7 * len(keys)) #     70% for the training set
cutoff2 = round(0.85 * len(keys)) #   15% for the validation set and 15% for the final test set

keystrain = keys[:cutoff]
keystest = keys[cutoff:cutoff2]
keysfinaltest = keys[cutoff2:]

print("cutoff " + str(cutoff))
print("cutoff2 " + str(cutoff2))


with open(data_path + mode + '_dataset_keystrain', 'wb') as fp:
    pickle.dump(keystrain, fp)
with open(data_path + mode + '_dataset_keystest', 'wb') as fp:
    pickle.dump(keystest, fp)
with open(data_path + mode + '_dataset_keysfinaltest', 'wb') as fp:
    pickle.dump(keysfinaltest, fp)

TrainX = []
TrainY = []
ValidateX = []
ValidateY = []
FinaltestX = []
FinaltestY = []


words = list(w2v_model.wv.index_to_key)
codes, lables = [], []
for code, lable in allblocks:
    codes.append(code.split(' '))
    lables.append(lable)
Tokenization(codes)
total_sequences, word_index = LoadTokenizer(codes)
word_vectors = w2v_model.wv
words = list(w2v_model.wv.index_to_key)
embedding_matrix = np.zeros((len(word_index) + 1, 200))#word vectors has dimention 200
for word, i in word_index.items():
    embedding_vector = word_vectors[words[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
weights=torch.from_numpy(embedding_matrix)
with open(embedding_path + mode + '_weights', 'wb') as fp:
    pickle.dump(weights, fp)
    
max_length = fulllength 
total_sequences = pad_sequences(total_sequences, maxlen=max_length)

print(total_sequences.shape)
print("Creating training dataset... (" + mode + ")")
for k in keystrain:
    TrainX.append(total_sequences[k]) #append the list of vectors to the X (independent variable)
    TrainY.append(lables[k]) #append the label to the Y (dependent variable)
print("Train length: " + str(len(TrainX)))
print('saving datasets')
with open(data_path + 'plain_' + mode + '_dataset-train-X_'+ "word2vec_"+w+str(mincount) + "-" + str(iterationen) +"-" + str(s) + "__" + mode2, 'wb') as fp:
    pickle.dump(TrainX, fp)
with open(data_path + 'plain_' + mode + '_dataset-train-Y_'+ "word2vec_"+w+str(mincount) + "-" + str(iterationen) +"-" + str(s) + "__" + mode2, 'wb') as fp:
    pickle.dump(TrainY, fp)
now = datetime.now() # current date and time
nowformat = now.strftime("%H:%M")
print("time: ", nowformat)

print("Creating validation dataset...")
for k in keystest:
    ValidateX.append(total_sequences[k]) #append the list of vectors to the X (independent variable)
    ValidateY.append(lables[k]) #append the label to the Y (dependent variable)
print("Test length: " + str(len(ValidateX)))
with open(data_path + 'plain_' + mode + '_dataset-validate-X_'+ "word2vec_"+w+str(mincount) + "-" + str(iterationen) +"-" + str(s) + "__" + mode2, 'wb') as fp:
    pickle.dump(ValidateX, fp)
with open(data_path + 'plain_' + mode + '_dataset-validate-Y_'+ "word2vec_"+w+str(mincount) + "-" + str(iterationen) +"-" + str(s) + "__" + mode2, 'wb') as fp:
    pickle.dump(ValidateY, fp)
now = datetime.now() # current date and time
nowformat = now.strftime("%H:%M")
print("time: ", nowformat)

print("Creating finaltest dataset...")
for k in keysfinaltest:
    FinaltestX.append(total_sequences[k]) #append the list of vectors to the X (independent variable)
    FinaltestY.append(lables[k]) #append the label to the Y (dependent variable)
print("Finaltesting length: " + str(len(FinaltestX)))
with open(data_path + mode + '_dataset_finaltest_X', 'wb') as fp:
    pickle.dump(FinaltestX, fp)
with open(data_path + mode + '_dataset_finaltest_Y', 'wb') as fp:
    pickle.dump(FinaltestY, fp)
now = datetime.now() # current date and time
nowformat = now.strftime("%H:%M")
print("time: ", nowformat)

print("saved dataset")


In [ ]:
import sys
sys.path.append('..')
from src.includes import *#Prepare the data for models
model_name = 'plstm'
home_path = '/home/swj/VD/'
embedding_path = home_path + 'embedding/'

#default mode / type of vulnerability
mode = "remote_code_execution"
data_path = home_path + 'data/python_data_set/' + mode + '/'
### hyperparameters for the w2v model
mincount = 10 #minimum times a word has to appear in the corpus to be in the word2vec model
iterationen = 300 #training iterations for the word2vec model
s = 200 #dimensions of the word2vec model
w = "withString" #word2vec model is not replacing strings but keeping them
### paramters for the filtering and creation of samples
restriction = [20000,5,6,10] #which samples to filter out
step = 5 #step lenght n in the description
fulllength = 200 #context length m in the description
mode2 = str(step)+"_"+str(fulllength) 

Train_X, Train_y, ValidateX, ValidateY = LoadPickleData(data_path + 'plain_' + mode + '_dataset-train-X_'+ "word2vec_"+w+str(mincount) + "-" + str(iterationen) +"-" + str(s) + "__" + mode2), \
LoadPickleData(data_path + 'plain_' + mode + '_dataset-train-Y_'+ "word2vec_"+w+str(mincount) + "-" + str(iterationen) +"-" + str(s) + "__" + mode2), \
LoadPickleData(data_path + 'plain_' + mode + '_dataset-validate-X_'+ "word2vec_"+w+str(mincount) + "-" + str(iterationen) +"-" + str(s) + "__" + mode2), \
LoadPickleData(data_path + 'plain_' + mode + '_dataset-validate-Y_'+ "word2vec_"+w+str(mincount) + "-" + str(iterationen) +"-" + str(s) + "__" + mode2)

X_train =  np.array(Train_X)
y_train =  np.array(Train_y)
X_test =  np.array(ValidateX)
y_test =  np.array(ValidateY)

#in the original collection of data, the 0 and 1 were used the other way round, so now they are switched so that "1" means vulnerable and "0" means clean.

for i in range(len(y_train)):
    if y_train[i] == 0:
        y_train[i] = 1
    else:
        y_train[i] = 0
    
for i in range(len(y_test)):
    if y_test[i] == 0:
        y_test[i] = 1
    else:
        y_test[i] = 0



now = datetime.now() # current date and time
nowformat = now.strftime("%H:%M")
print("numpy array done. ", nowformat)

print(str(len(X_train)) + " samples in the training set.")      
print(str(len(X_test)) + " samples in the validation set.") 
  
csum = 0
for a in y_train:
    csum = csum+a
print("percentage of vulnerable samples: "  + str(int((csum / len(X_train)) * 10000)/100) + "%")
  
testvul = 0
for y in y_test:
    if y == 1:
        testvul = testvul+1
print("absolute amount of vulnerable samples in test set: " + str(testvul))

  

now = datetime.now() # current date and time
nowformat = now.strftime("%H:%M")
print("Starting DPLSTM: ", nowformat)


print(X_train.shape, X_test.shape)

train_set_x = torch.Tensor(X_train).long().to(device) # transform to torch tensor
train_set_y = torch.Tensor(y_train).to(device)
train_dataset = TensorDataset(train_set_x,train_set_y) # create your datset
train_loader = DataLoader(train_dataset, batch_size=512) # create your dataloader

test_set_x = torch.Tensor(X_test).long().to(device) # transform to torch tensor
test_set_y = torch.Tensor(y_test).to(device)
test_dataset = TensorDataset(test_set_x,test_set_y) # create your datset
test_loader = DataLoader(test_dataset, batch_size=512) # create your dataloader

In [ ]:
import sys
sys.path.append('..')
from src.includes import *

model_name = 'dlstm'
home_path = '/home/swj/VD/'
code_type = 'python'
embedding_path = home_path + 'embedding/'
mode = "remote_code_execution"
model_path = home_path + 'trained_model/' + code_type + '_' + mode + '/'

data_path = home_path + 'data/python_data_set/' + mode + '/'
mincount = 10 #minimum times a word has to appear in the corpus to be in the word2vec model
iterationen = 300 #training iterations for the word2vec model
s = 200 #dimensions of the word2vec model
w = "withString" #word2vec model is not replacing strings but keeping them
restriction = [20000,5,6,10] #which samples to filter out
step = 5 #step lenght n in the description
fulllength = 200 #context length m in the description
mode2 = str(step)+"_"+str(fulllength) 
recording_type = 'loss&acc'

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

#torch.cuda.set_per_process_memory_fraction(0.5, 0)
print(device)

weights=LoadPickleData(embedding_path + mode + '_weights')

dlstm = DLSTM_python_net(weights).to(device)
plstm = PLSTM_python_net(weights).to(device)
blstm = BLSTM_python_net(weights).to(device)
lstm = LSTMnet(weights).to(device)


criterion = nn.BCEWithLogitsLoss()
optimizer = None
if model_name == 'blstm':
    optimizer = optim.Adam(blstm.parameters(), lr=0.001)
elif model_name == 'dlstm':
    optimizer = optim.Adam(dlstm.parameters(), lr=0.001)
elif model_name == 'plstm':
    optimizer = optim.Adam(plstm.parameters(), lr=0.001)
elif model_name == 'lstm':
    optimizer = optim.Adam(lstm.parameters(), lr=0.001)

epochs = 60
is_first = True
def train(model, device, train_loader, test_loader, optimizer, epoch):
    model.train()
    train_loss, train_acc = 0, 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.unsqueeze(1))
        acc = binary_acc(output, target.unsqueeze(1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += acc.item()
    model.eval()     # Optional when not using Model Specific layer
    valid_loss, valid_acc = 0, 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = criterion(output, target.unsqueeze(1))
        acc = binary_acc(output, target.unsqueeze(1))
        valid_loss += loss.item() 
        valid_acc += acc.item() 

    print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(test_loader)}')
    return train_loss, train_acc, valid_loss, valid_acc

def D_train(model, device, train_loader, test_loader, optimizer, epoch, A):
    model.train()
    train_loss, train_acc = 0, 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data, A)
        loss = criterion(output, target.unsqueeze(1))
        acc = binary_acc(output, target.unsqueeze(1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += acc.item()
    
    model.eval()     # Optional when not using Model Specific layer
    valid_loss, valid_acc = 0, 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data, A)
        loss = criterion(output, target.unsqueeze(1))
        acc = binary_acc(output, target.unsqueeze(1))
        valid_loss += loss.item() 
        valid_acc += acc.item() 

    print(f'Epoch {epoch+1} \t\t Training Loss: {train_loss / len(train_loader)} \t\t Validation Loss: {valid_loss / len(test_loader)}')
    return train_loss, train_acc, valid_loss, valid_acc


writer = SummaryWriter(log_dir=home_path + 'src/runs/' + model_name + '_' + code_type + '_' + mode + '/', comment=model_name + '_' + code_type + '_' + mode)
csv_writer = []

ts = torch.rand(train_set_x.shape, dtype=torch.double).to(device)
domain_set = len(train_set_x) * [1] + len(ts) * [0]
A = torch.eye(200, dtype = train_set_x.dtype).to(device)
for epoch in range(epochs):
    train_set_x.to(device)
    train_set_x = torch.tensor(train_set_x, dtype=torch.double)
    A = Newton(torch.cat((train_set_x, ts), dim = 0), domain_set, 4)
    ts = ts @ A.sqrt()

    time1 = time.time()
    if model_name == 'blstm':
        train_loss, train_acc, valid_loss, valid_acc = train(blstm, device, train_loader, test_loader, optimizer, epoch)
    elif model_name == 'dlstm':
        train_loss, train_acc, valid_loss, valid_acc = train(dlstm, device, train_loader, test_loader, optimizer, epoch)
    elif model_name == 'plstm':
        train_loss, train_acc, valid_loss, valid_acc = D_train(plstm, device, train_loader, test_loader, optimizer, epoch, A)
    elif model_name == 'lstm':
        train_loss, train_acc, valid_loss, valid_acc = train(lstm, device, train_loader, test_loader, optimizer, epoch)
    time2 = time.time()
    writer.add_scalar('Loss/train', train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)
    writer.add_scalar('Loss/valid', valid_loss, epoch)
    writer.add_scalar('Accuracy/valid', valid_acc, epoch)
    csv_writer.append([epoch, train_loss, valid_loss])
    print(f'Epoch {epoch} elapse {time2 - time1}s')

Columns = ['epoch','train_loss','valid_loss']
CSV_writer=pd.DataFrame(columns=Columns,data=csv_writer)
print(CSV_writer)
csv_saved_path = home_path + 'results/' + model_name + '/' + code_type + '/' + mode + '/'
if not os.path.exists(csv_saved_path): os.makedirs(csv_saved_path)
CSV_writer.to_csv(csv_saved_path + model_name + '_' + mode + '.csv',encoding='gbk')

if not os.path.exists(model_path): os.makedirs(model_path)
if model_name == 'blstm':
    torch.save(blstm.state_dict(),  model_path + model_name + '_model.h5')
elif model_name == 'dlstm':
    torch.save(dlstm.state_dict(), model_path + model_name + '_model.h5')
elif model_name == 'lstm':
    torch.save(lstm.state_dict(), model_path + model_name + '_model.h5')
elif model_name == 'plstm':
    torch.save(plstm.state_dict(), model_path + model_name + '_model.h5')

In [ ]:
import sys
sys.path.append('..')

from src.includes import *


model_name = 'plstm'
home_path = '/home/swj/VD/'
code_type = 'python'
embedding_path = home_path + 'embedding/'
mode = "remote_code_execution"
ways = 'myway'
model_path = home_path + 'trained_model/' + code_type + '_' + mode + '/'

#default mode / type of vulnerability
data_path = home_path + 'data/python_data_set/' + mode + '/'
repre_saved_path = home_path + 'Representation/' + model_name + '_' + ways + '/' + code_type + '/' + mode + '/'

### hyperparameters for the w2v model
mincount = 10 #minimum times a word has to appear in the corpus to be in the word2vec model
iterationen = 300 #training iterations for the word2vec model
s = 200 #dimensions of the word2vec model
w = "withString" #word2vec model is not replacing strings but keeping them
### paramters for the filtering and creation of samples
restriction = [20000,5,6,10] #which samples to filter out
step = 5 #step lenght n in the description
fulllength = 200 #context length m in the description
mode2 = str(step)+"_"+str(fulllength) 

Test_X, Test_y = LoadPickleData(data_path + mode + '_dataset_finaltest_X'), LoadPickleData(data_path + mode + '_dataset_finaltest_Y')

X_test =  np.array(Test_X)
X_test = torch.Tensor(X_test).long().to(device) # transform to torch tensor

    
for i in range(len(Test_y)):
    if Test_y[i] == 0:
        Test_y[i] = 1
    else:
        Test_y[i] = 0
        
#Train_X_repre, Train_y_repre, Test_X_repre, Test_y_repre = [], [], [], []
#obtain representations
def ObtainRepresentations_by_batch_size(input_sequences, feature_model, BATCH_SIZE):
    num_batches_per_epoch = int((len(input_sequences) - 1) / BATCH_SIZE) + 1
    data_size = len(input_sequences)
    representations_total = []
    model = feature_model
    for batch_num in range(num_batches_per_epoch):
        start_index = batch_num * BATCH_SIZE
        end_index = min((batch_num + 1) * BATCH_SIZE, data_size)
        print ("-------start_index------------")
        print (start_index)
        print ("-------end_index------------")
        print (end_index)
        print(input_sequences[start_index: end_index].shape)
        representations = model(input_sequences[start_index: end_index])
        representations_total = representations_total + representations.tolist()
    return np.asarray(representations_total)

weights=LoadPickleData(embedding_path + mode + '_weights')

if model_name == 'dlstm':
    net = DLSTM_python_net(weights).to(device)
elif model_name == 'plstm':
    net = PLSTM_python_net(weights).to(device)
elif model_name == 'blstm':
    net = BLSTM_python_net(weights).to(device)
    
#change this

net.load_state_dict(torch.load(model_path + model_name + '_model.h5'))
net.eval()

feature_model = None
if model_name == 'blstm':
    feature_model = BLSTM_python_Feature_Net(net, weights).to(device)
elif model_name == 'dlstm':
    feature_model = DLSTM_python_Feature_Net(net, weights).to(device)
elif model_name == 'plstm':
    feature_model = PLSTM_python_Feature_Net(net, weights).to(device)
    
feature_model.eval()
start_time = time.time()
if not os.path.exists(repre_saved_path): os.makedirs(repre_saved_path)

obtained_repre = ObtainRepresentations_by_batch_size(X_test, feature_model, 512)
SavedPickle(repre_saved_path + "X_test.pkl", obtained_repre)
SavedPickle(repre_saved_path + "y_test.pkl", Test_y)
print("Saving the obtained representations....") 
end_time = time.time()
print('uisng' + str(end_time - start_time) + 's')
print("The obtained representations are saved in: " + str(repre_saved_path) + ".")

In [ ]:
#calculating the num of datasets
import sys
sys.path.append('..')
from src.includes import *

model_name = 'bplstm'
home_path = '/home/swj/VD/'
embedding_path = home_path + 'embedding/'

#default mode / type of vulnerability
mode = "command_injection"

data_sets_names = ['command_injection', 'open_redirect', 'path_disclosure', 'remote_code_execution', 'sql', 'xsrf', 'xss']
for data_sets_name in data_sets_names:
    data_path = home_path + 'data/python_data_set/' + data_sets_name + '/'
    train = LoadPickleData(data_path + 'plain_' + data_sets_name + '_dataset-train-Y_word2vec_withString10-300-200__5_200')
    validation = LoadPickleData(data_path + 'plain_' + data_sets_name + '_dataset-validate-Y_word2vec_withString10-300-200__5_200')
    test = LoadPickleData(data_path + data_sets_name + '_dataset_finaltest_Y')
    
    tot = len(train + validation + test)
    nonvul = sum(train + validation + test)
    vul = tot - nonvul
    proportion = vul / tot
    print('the data_set is:  ' + data_sets_name + '    there are   ' + str(tot) + ' total samples,  ' + str(vul) + ' vul samples,  ' + str(nonvul) + ' nonvul samples,   ' + 'the propotion is:   ' + str(vul / tot))